import zipfile

zip_ref = zipfile.ZipFile("all.zip", 'r')
zip_ref.extractall(".")
zip_ref.close()

zip_ref = zipfile.ZipFile("train.zip", 'r')
zip_ref.extractall(".")
zip_ref = zipfile.ZipFile("test.zip", 'r')
zip_ref.extractall(".")
zip_ref.close()

In [43]:
from keras.models import Model
from keras.layers import Dense, Input, GlobalAveragePooling2D, Dropout, Lambda
from keras.applications import inception_resnet_v2
# from keras.preprocessing.image import *

import numpy as np
from tqdm import tqdm
import cv2

In [44]:
n = 25000
img_height, img_width = 299, 299
X = np.zeros((n, img_height, img_width, 3), dtype=np.uint8)
y = np.zeros((n, 1), dtype=np.uint8)

for i in tqdm(range(12500)): # n/2
    X[i] = cv2.resize(cv2.imread('train/cat.%d.jpg' % i), (img_height, img_width))
    X[i+12500] = cv2.resize(cv2.imread('train/dog.%d.jpg' % i), (img_height, img_width)) # n/2

# 打上标签
y[12500:] = 1

100%|██████████| 12500/12500 [00:57<00:00, 218.75it/s]


In [45]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)

In [46]:
i_input_tensor = Input((img_height, img_width, 3))
# x_input_tensor = inception_resnet_v2.preprocess_input(i_input_tensor)	# 错误的处理
x_input_tensor = Lambda(inception_resnet_v2.preprocess_input)(i_input_tensor) # 建立模型预处理

# 搭建基础模型
base_model = inception_resnet_v2.InceptionResNetV2(input_tensor=x_input_tensor, weights='imagenet', include_top=False)

范例：不运行
from keras.callbacks import ModelCheckpoint

'''
如果验证损失下降， 那么在每个训练轮之后保存模型。
'''
checkpointer = ModelCheckpoint(filepath='weights.hdf5', verbose=1, save_best_only=True)
model.fit(x_train, y_train, batch_size=128, epochs=20, verbose=0, validation_data=(X_test, Y_test), callbacks=[checkpointer])

### 下面修改

In [47]:
from keras.callbacks import ModelCheckpoint

checkpointer = ModelCheckpoint(filepath='weights_checkpint_030_770.hdf5', verbose=1, save_best_only=True)

### 下面修改

In [48]:
# 锁定所有神经网络层
for layers in base_model.layers:
    layers.trainable = False

i_output = GlobalAveragePooling2D()(base_model.output)
i_output = Dropout(0.3)(i_output) # 0.25 比 0.5 好
i_predictions = Dense(1, activation='sigmoid')(i_output)
model = Model(base_model.input, i_predictions)

model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [49]:
model.fit(X_train, y_train, batch_size=20,     # defalt 16
	epochs=8, 	# defalt 5
	validation_data=(X_valid, y_valid),
         verbose=1, callbacks=[checkpointer])


Train on 20000 samples, validate on 5000 samples
Epoch 1/8
20000/20000 [==============================] - 150s 8ms/step - loss: 0.1396 - acc: 0.9518 - val_loss: 0.0602 - val_acc: 0.9822

Epoch 00001: val_loss improved from inf to 0.06018, saving model to weights_checkpint_030_770.hdf5
Epoch 2/8
20000/20000 [==============================] - 133s 7ms/step - loss: 0.0915 - acc: 0.9671 - val_loss: 0.0448 - val_acc: 0.9880

Epoch 00002: val_loss improved from 0.06018 to 0.04482, saving model to weights_checkpint_030_770.hdf5
Epoch 3/8
20000/20000 [==============================] - 133s 7ms/step - loss: 0.0873 - acc: 0.9669 - val_loss: 0.0764 - val_acc: 0.9784

Epoch 00003: val_loss did not improve from 0.04482
Epoch 4/8
20000/20000 [==============================] - 133s 7ms/step - loss: 0.0861 - acc: 0.9666 - val_loss: 0.0914 - val_acc: 0.9734

Epoch 00004: val_loss did not improve from 0.04482
Epoch 5/8
20000/20000 [==============================] - 133s 7ms/step - loss: 0.0833 - acc: 0.

In [50]:
for i in range(len(model.layers)):
    print(model.layers[i].name, i)

input_4 0
lambda_4 1
conv2d_610 2
batch_normalization_610 3
activation_610 4
conv2d_611 5
batch_normalization_611 6
activation_611 7
conv2d_612 8
batch_normalization_612 9
activation_612 10
max_pooling2d_13 11
conv2d_613 12
batch_normalization_613 13
activation_613 14
conv2d_614 15
batch_normalization_614 16
activation_614 17
max_pooling2d_14 18
conv2d_618 19
batch_normalization_618 20
activation_618 21
conv2d_616 22
conv2d_619 23
batch_normalization_616 24
batch_normalization_619 25
activation_616 26
activation_619 27
average_pooling2d_4 28
conv2d_615 29
conv2d_617 30
conv2d_620 31
conv2d_621 32
batch_normalization_615 33
batch_normalization_617 34
batch_normalization_620 35
batch_normalization_621 36
activation_615 37
activation_617 38
activation_620 39
activation_621 40
mixed_5b 41
conv2d_625 42
batch_normalization_625 43
activation_625 44
conv2d_623 45
conv2d_626 46
batch_normalization_623 47
batch_normalization_626 48
activation_623 49
activation_626 50
conv2d_622 51
conv2d_624 52

### 下面

In [51]:
# 放开靠后部分网络层进行 fine tune 微调模型
for layer in model.layers[770:]:	# total 783 layers
    layer.trainable = True


from keras import optimizers
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])


In [52]:
model.fit(X_train, y_train, batch_size=20, 	# first: 16
	epochs=8, 	# first: 5 
	validation_data=(X_valid, y_valid),
         verbose=1, callbacks=[checkpointer])

Train on 20000 samples, validate on 5000 samples
Epoch 1/8
20000/20000 [==============================] - 166s 8ms/step - loss: 0.0693 - acc: 0.9753 - val_loss: 0.0416 - val_acc: 0.9902

Epoch 00001: val_loss improved from 0.04482 to 0.04160, saving model to weights_checkpint_030_770.hdf5
Epoch 2/8
20000/20000 [==============================] - 149s 7ms/step - loss: 0.0726 - acc: 0.9722 - val_loss: 0.0414 - val_acc: 0.9908

Epoch 00002: val_loss improved from 0.04160 to 0.04142, saving model to weights_checkpint_030_770.hdf5
Epoch 3/8
20000/20000 [==============================] - 149s 7ms/step - loss: 0.0732 - acc: 0.9724 - val_loss: 0.0398 - val_acc: 0.9908

Epoch 00003: val_loss improved from 0.04142 to 0.03980, saving model to weights_checkpint_030_770.hdf5
Epoch 4/8
20000/20000 [==============================] - 149s 7ms/step - loss: 0.0702 - acc: 0.9723 - val_loss: 0.0389 - val_acc: 0.9908

Epoch 00004: val_loss improved from 0.03980 to 0.03886, saving model to weights_checkpint_

### 下面修改

In [53]:
model.save('modelInceptionResNet_transfer_Doup030_770layer.h5')

读取测试图片数据

In [54]:
X_test = np.zeros((n, img_height, img_width, 3), dtype=np.uint8)
for i in tqdm(range(12500)):
    j = i+1
    X_test[i] = cv2.resize(cv2.imread('test/%d.jpg' % j), (img_height, img_width))

100%|██████████| 12500/12500 [00:28<00:00, 432.46it/s]


In [55]:
y_pred = model.predict(X_test, verbose=1)
y_pred = y_pred.clip(min=0.005, max=0.995)

25000/25000 [==============================] - 154s 6ms/step


### 下面修改

In [56]:
import pandas as pd
df = pd.read_csv("sample_submission.csv")

for i in range(12500):
	df.set_value(i, 'label', y_pred[i])

df.to_csv('pred_changePredictCsvMethond_Drop030_770layer.csv', index=None)

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:5: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
